In [1]:
import PIL
from PIL import Image
from numpy import asarray
import nrrd

import numpy as np
import pandas as pd
import os
from radiomics import featureextractor
import radiomics
import csv

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, KFold, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_extraction import image
from skimage import *

import matplotlib.pyplot as plt
import statistics
import collections
import pickle

**Image Chooser**

In [2]:
#normalization function
def mean_norm(df):
    return df.apply(lambda x: (x-x.mean())/ x.std(), axis=0)

In [3]:
def radiomics_features(im_PATH, mask_dir):
    '''
    
        ------------------------------------------
        Parameters:

        im_PATH: directory to all the images
        mask_dir: directory to all the pre-made masks

        ------------------------------------------
        Returns:

        feature_matrix: feature of all the masks of an image

        ------------------------------------------
        
    '''
 
    os.chdir(im_PATH) #change to image directory
    list_of_photos = os.listdir(im_PATH) #store all items in directory in a list
    feature_matrix_masks = pd.DataFrame()
    for c,i in enumerate(list_of_photos): 
        if ("count" in i) or (i == '.DS_Store'): #do not look at these 
            continue

        photo_dir = im_PATH + i #full image file path
        radiomics.setVerbosity(level=40)
        
         #initialize extractor
        feature_extractor = featureextractor.RadiomicsFeatureExtractor(geometryTolerance = True)
        list_of_masks = os.listdir(mask_dir)
        
        for j in list_of_masks:
            #extract features and store into result
            result = feature_extractor.execute(photo_dir, mask_dir + j)
            
            #put all the extracted features in a dataframe
            feature_row = pd.DataFrame.from_dict(result, orient = 'index')
            feature_row = feature_row.transpose()
            
            #subset useful columns
            feature_row = feature_row.iloc[:, 24:]
            
            #add identifiers
            feature_row.insert(0, 'name', [i.split(".jpg")[0]])
            feature_row.insert(1, 'image', [j.split(".nrrd")[0]])
            
            #append row to feature_matrix
            feature_matrix_masks = feature_matrix_masks.append(feature_row, ignore_index = True)

    return feature_matrix_masks



In [4]:
def logreg_classifier(im_PATH, mask_PATH, im_type, cwd): 
    '''
    ------------------------------------------
    Parameters:
    
    im_PATH: directory to all the images
    mask_PATH: directory to all pre-made masks
    im_type ('nerve' or 'cell'): tells whether we are training to identify a good image for nerve 
             segementation or cell density calculation
    cwd: current working directory
        
    ------------------------------------------
    Returns:
    
    logreg: logistic regression classifier model 

    ------------------------------------------
    
    ''' 
    #import labels/targets
    if im_type == 'nerve':
        label_PATH = cwd + '/Nerve Segmentation w Annotations/nerve_training_labels.csv'
    else:
        label_PATH = cwd + '/Cell Density Data w Annotations/cell_training_labels.csv'

    #extract features
    feature_matrix = radiomics_features(im_PATH, mask_PATH)
    
    #concatenate name and image #
    feature_matrix['name'] = feature_matrix['name'] + '-' + feature_matrix['image']
    feature_matrix.drop(['image'], axis = 1, inplace = True)
    
    #read in labels
    y = pd.read_csv(label_PATH)
    y = y.astype({'image':'string'})
    y['name'] = y['name'] + '-' + y['image']
    y.drop(['image'], axis = 1, inplace = True)

    #adjust y's rows based on the names of feature_matrix
    y = y.set_index('name')
    y = y.reindex(index = feature_matrix['name'])
    y = y.reset_index()
    
    #store names
    names = feature_matrix['name']

    #normalize
    X = mean_norm(feature_matrix.drop(['name'], axis=1))
    X['name'] = names

    #initialize logistic regression model
    logreg = LogisticRegression(max_iter = 1000, penalty = 'l1', solver = 'liblinear')
    
    #prep training data
    X_train = X.drop(['name'], axis = 1)
    y_train = y['label']
    
    #train
    logreg.fit(X_train, y_train)

    return logreg

In [5]:
def photo_chooser(logreg, im_dir, mask_PATH):
    '''
    ------------------------------------------
    Parameters:
    
    logreg: logistic regression model
    im_dir: directory to evaluation images
    mask_PATH: directory to all the masks
        
    ------------------------------------------
    Returns:
    
    PATHS: list of paths to the chosen images
        
    ------------------------------------------
    
    ''' 
    #extract features
    feature_matrix = radiomics_features(im_dir, mask_PATH)
    
    #get predicted probabilities of whether it is a 0 or 1 on normalized testing feature matrix
    y_pred_prob = logreg.predict_proba(mean_norm(feature_matrix.drop(['name', 'image'], axis=1)))

    #get all the predicted probabilities of it being a 1
    one_prob = np.array([prob[1] for prob in y_pred_prob])
    
    #extract indices of the elements in the 75th percentile
    percentile_75 = np.where(np.array(one_prob >= np.percentile(one_prob,75)) == True)[0]
    
    PATHS = []
    #get paths of all images in 75th percentile
    for i in percentile_75:
        path = im_dir + np.array(feature_matrix['name'])[i]+'.jpg'
        PATHS.append(path)
    
    counts = collections.Counter(PATHS)
    
    return [x[0] for x in counts.most_common(3)]

In [6]:
def validation(logreg, im_dir_ls, mask_PATH):
    '''
    ------------------------------------------
    Parameters:
    
    logreg: logistic regression model
    im_dir_ls: list of image directores
    mask_PATH: directory to all the masks
        
    ------------------------------------------
    Returns:
    
        NA
    ------------------------------------------
    
    '''
    #initialize list of errors
    errors = []
    #loop through all the image directories
    for im_dir in im_dir_ls:

        good_im = []
        #loop thorugh all the images in the directory
        for im in os.listdir(im_dir):
            #collect all the good images from each image directory
            if 'count' in im:
                good_im.append(int(im.split('.')[0]))
        
        #selected good images by models
        selected = photo_chooser(logreg, im_dir, mask_PATH)
        
        #get image name    
        im_selected = [int((i[-7:]).split('.')[0]) for i in selected]
        
        #find minimum error between selected image and closest chosen image by clinician
        for i in im_selected:
            errors.append(min([(abs(i-k)-1) for k in good_im]))
    
    #plot MAE figures
    fig = plt.figure(figsize = (10,7))
    plt.boxplot(errors)
    plt.title('MAE of distance from clinically chosen images')
    plt.show()
    
    print('MAE: ', statistics.mean(errors))

### Basal Cell Density Image Model

In [11]:
os.chdir("/Users/Emily/Documents/BioE 223A")
cwd = os.getcwd()

In [8]:
# cell_logreg = logreg_classifier(cwd + "/Cell Density/", cwd + "/Masks/", "cell", cwd)

In [13]:
cell_logreg = pickle.load(open('cell_logreg.sav', 'rb'))

In [24]:
photo_chooser(cell_logreg, cwd + '/Data/Central BCD data/Moderate/50-od-9/', cwd + "/Masks/")
dir_ls = [cwd + '/Data/Central BCD data/Moderate/4-od-18/', 
          cwd +'/Data/Central BCD data/Mild/12-od-0/', 
          cwd + '/Data/Central BCD data/Control/N1-od-1/',
          cwd + '/Data/Central BCD data/Severe/14-od-6/', 
          cwd + '/Data/Central BCD data/Severe/39-od-13/']
print('CELL DENSITY MODEL')
validation(cell_logreg, dir_ls, cwd + "/Masks/")

### Nerve Density Image Model

In [18]:
os.chdir("/Users/Emily/Documents/BioE 223A")
cwd = os.getcwd()

In [19]:
nerve_logreg = pickle.load(open('nerve_logreg.sav', 'rb'))

In [ ]:
# nerve_logreg = logreg_classifier(cwd + "/Nerve Segmentation/", cwd + "/Masks/", "nerve", cwd)

In [23]:
photo_chooser(nerve_logreg, cwd + '/Data/Basal nerve density data/Moderate/38-os-0/', cwd + "/Masks/")

dir_ls = [cwd + '/Data/Basal nerve density data/Control/N1-od-0/', 
          cwd + '/Data/Basal nerve density data/Mild/12-od-4/', 
          cwd + '/Data/Basal nerve density data/Moderate/10-os/', 
          cwd + '/Data/Basal nerve density data/Moderate/70-os-52/',
          cwd + '/Data/Basal nerve density data/Mild/12-od-4/']
print('\nNERVE SEGMENTATION MODEL')
validation(nerve_logreg, dir_ls, cwd + "/Masks/")